In [1]:
from graph_tool import Graph
import pandas as pd
import numpy as np
import datetime
import netpixi
from netpixi.data.gt import *
import seaborn as sns
from graph_tool import clustering

In [3]:
# Carrega dados limpos para arestas
arestas = pd.read_csv("arestas.csv", index_col=0)
arestas

,T_min,Freq,RelCost,Dist,CITY_STATE_A,CITY_STATE_B
0,2160,7.5,0.023148,570.727977,Manaus - AM,Uarini - AM
1,8640,3.0,0.026620,787.834843,Japurá - AM,Manaus - AM
2,375,42.0,0.028000,279.152113,Figueirópolis D'Oeste - MT,Várzea Grande - MT
3,1920,7.5,0.031250,532.505675,Alvarães - AM,Manaus - AM
4,1380,6.0,0.036232,313.315756,Alvarães - AM,Codajás - AM
...,...,...,...,...,...,...
43838,30,70.0,0.186667,11.597523,Pedrinópolis - MG,Santa Juliana - MG
43839,148,7.0,0.066216,62.540653,Croatá - CE,Ubajara - CE
43840,240,14.0,0.136500,153.413295,Presidente Prudente - SP,Mandaguaçu - PR
43841,60,21.0,0.159500,34.112975,Mandaguaçu - PR,Marialva - PR


In [4]:
# Carrega dados limpos para vertices
vertices = pd.read_csv("vertices.csv", index_col=0)
vertices

,IBGE_DU,IBGE_DU_URBAN,IBGE_POP,IBGE_15-59,IDHM,GDP,GDP_CAPITA,CITY_STATE
0,4655.0,3233.0,10656.0,6896.0,0.690,287984.49,15628.40,Abadiânia - GO
1,7694.0,6667.0,18464.0,11979.0,0.698,430235.36,18250.42,Abaeté - MG
2,31061.0,19057.0,82956.0,53516.0,0.628,1249255.29,8222.36,Abaetetuba - PA
3,2791.0,1251.0,4538.0,2631.0,0.628,73151.46,6370.41,Abaiara - CE
4,2572.0,1193.0,3725.0,2319.0,0.603,64324.59,6982.70,Abaíra - BA
...,...,...,...,...,...,...,...,...
4084,11637.0,8461.0,32497.0,19371.0,0.585,342261.06,7089.97,Xique-Xique - BA
4085,673.0,489.0,1469.0,874.0,0.623,19582.06,8793.02,Zabelê - PB
4086,799.0,638.0,1836.0,1165.0,0.730,75934.89,29149.67,Zacarias - SP
4087,11886.0,7611.0,30803.0,18136.0,0.595,380509.30,7489.46,Zé Doca - MA


In [7]:
# Cria grafo
grafo_conexoes = Graph(directed=False) # não-dirigido

In [8]:
#propriedades dos vertices
grafo_conexoes.vp['CITY_STATE'] = grafo_conexoes.new_vp('object')
grafo_conexoes.vp['IBGE_DU'] = grafo_conexoes.new_vp('float')
grafo_conexoes.vp['IBGE_DU_URBAN'] = grafo_conexoes.new_vp('float')
grafo_conexoes.vp['IBGE_POP'] = grafo_conexoes.new_vp('float')
grafo_conexoes.vp['IBGE_15-59'] = grafo_conexoes.new_vp('float')
grafo_conexoes.vp['IDHM'] = grafo_conexoes.new_vp('float')
grafo_conexoes.vp['GDP'] = grafo_conexoes.new_vp('float')
grafo_conexoes.vp['GDP_CAPITA'] = grafo_conexoes.new_vp('float')
#propriedades dos arestas
grafo_conexoes.ep['T_min'] = grafo_conexoes.new_ep('int')
grafo_conexoes.ep['Freq'] = grafo_conexoes.new_ep('float')
grafo_conexoes.ep['RelCost'] = grafo_conexoes.new_ep('float')
grafo_conexoes.ep['Dist'] = grafo_conexoes.new_ep('float')

In [9]:
# Cria numero de vertices
grafo_conexoes.add_vertex(vertices.shape[0])

<generator object Graph.add_vertex.<locals>.<genexpr> at 0x7fddaca67350>

In [10]:
# Adiciona propriedades aos vertices
for i, label in vertices.iterrows():
    for col, val in zip(label.index, label):
        grafo_conexoes.vp[col][grafo_conexoes.vertex(i)] = val

In [11]:
# Adiciona arestas e propriedades
for i, row in arestas.iterrows():
    de = row["CITY_STATE_A"]
    para = row["CITY_STATE_B"]
    indice_de = vertices.index[vertices["CITY_STATE"] == de][0]
    indice_para = vertices.index[vertices["CITY_STATE"] == para][0]
    grafo_conexoes.add_edge(indice_de, indice_para)
    label = row[["T_min", "Freq", "RelCost", "Dist"]]
    for col, val in zip(label.index, label):
            grafo_conexoes.ep[col][grafo_conexoes.edge(indice_de, indice_para)] = val

In [12]:
# Salva grafo
save_gt(grafo_conexoes, 'grafo_conexoes.net.gz')